In [1]:
import os
import time
from itertools import combinations, product

import numpy as np
import pandas as pd
from tqdm import tqdm

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express.colors as px_colors
import plotly.io as pio
from scipy.spatial.distance import squareform

import SpAM_Simulations.metrics as metrics
from SpAM_Simulations.experiment import ExperimentParameters
from SpAM_Simulations.simulation import load_latest_simulation, create_simulation

pio.renderers.default = 'browser'

## Load or Make Simulation

In [ ]:
try:
    sim = load_latest_simulation(os.getcwd())
except FileNotFoundError:

    # # small and simple simulation
    # sim = create_simulation(
    #     n_images = 50,
    #     n_dims = 10,
    #     num_subjects = [20, 30],
    #     trials_per_subject = [8, 10],
    #     images_per_trial = [16, 20],
    #
    #     subjects_noise_scale = [0.2, 0.8],     # relative to gt_distances.std()
    #     subjects_noise_df = [1, 5],
    #     reps = 3,
    #     seed = 42,
    #     verbose = True,
    # )
    #
    # # no-noise simulation
    # sim = create_simulation(
    #     n_images = 754,
    #     n_dims = 10,
    #     num_subjects = [20, 50],
    #     trials_per_subject = [10, 15],
    #     images_per_trial = [16, 25],
    #
    #     subjects_noise_scale = [0.0],     # relative to gt_distances.std()
    #     subjects_noise_df = [5],
    #     reps = 3,
    #     seed = 42,
    #     verbose = True,
    # )

    # the real, heavy simulation with all configurations
    sim = create_simulation(
        n_images = 754,
        n_dims = 10,
        num_subjects = [20, 30, 200],
        trials_per_subject = [10, 15],
        images_per_trial = [16, 20, 25],
        subjects_noise_scale = [0, 0.2, 0.5, 0.8],    # relative to gt_distances.std()
        subjects_noise_df = [1, 5],
        reps = 3,
        seed = 42,
        verbose = True,
    )

Running experiments:  25%|██▍       | 107/432 [01:03<11:21,  2.10s/it]

## Evaluate Simulation
### Coverage
We check three coverage scores for the simulated data:
1. **Image Coverage**: the percentage of images that were observed at least once across all subjects and trials.
2. **Pair Coverage**: the percentage of image-pairs that were observed at least once across all subjects and trials.
3. **P[_is connected_]**: the probability that the graph of observed image-pairs is fully connected (i.e. has only 1 connected component). We check this because the `MDS` algorithm requires a fully connected graph to run, so if we have a low probability of getting a fully connected graph for a given experimental configuration, that configuration may not be suitable for running MDS on.

In [ ]:
coverage = []
for param in sim._results.keys():
    for rep, result in enumerate(sim._results[param]):
        coverage.append(pd.Series(metrics.coverage(result)).rename((*param, rep)))
del param, rep, result

coverage = pd.concat(coverage, axis=1).T
coverage.index.names = [
    'n_subjects', 'trials_per_subject', 'images_per_trial', 'subjects_noise_scale', 'subjects_noise_df', 'rep',
]
coverage["is_connected"] = coverage["num_connected_components"] == 1.0

# coverage scores are not affected by subject noise, so we average across those parameters
coverage_summary = (
    coverage.groupby(["n_subjects", "trials_per_subject", "images_per_trial"])
    .agg(
        num_reps = ("img_coverage", "size"),
        percent_img_coverage_mean = ("img_coverage", "mean"),
        percent_img_coverage_sem = ("img_coverage", "sem"),
        percent_pair_coverage_mean = ("pair_coverage", "mean"),
        percent_pair_coverage_sem = ("pair_coverage", "sem"),
        p_is_connected_mean = ("is_connected", "mean"),
        p_is_connected_sem = ("is_connected", "sem"),
    )
    .sort_index()
    .reset_index()
)

In [ ]:
ROW_TITLES = {
    "% IMG COVERAGE": "percent_img_coverage",
    "% PAIR COVERAGE": "percent_pair_coverage",
    "P[CONNECTED]": "p_is_connected",
}
COL_TITLES = {k: f"{k} Images per Trial" for k in coverage_summary["images_per_trial"].unique()}
coverage_fig = make_subplots(
    rows=len(ROW_TITLES), cols=len(COL_TITLES),
    column_titles=list(COL_TITLES.values()),
    shared_xaxes=True, shared_yaxes=True,
    x_title="Number of Subjects",
    vertical_spacing=0.05, horizontal_spacing=0.025,
)
for c, (images_per_trial, col_title) in enumerate(COL_TITLES.items()):
    for r, (row_title, prefix) in enumerate(ROW_TITLES.items()):
        for i, tr in enumerate(coverage_summary["trials_per_subject"].unique()):
            color = px_colors.qualitative.Plotly[i]
            df = coverage_summary.loc[
                (coverage_summary["trials_per_subject"] == tr) & (coverage_summary["images_per_trial"] == images_per_trial),
            ]
            coverage_fig.add_trace(
                row=r + 1, col=c + 1, trace=go.Scatter(
                    x=df["n_subjects"],
                    y=df[f"{prefix}_mean"],
                    error_y=dict(type="data", array=df[f"{prefix}_sem"], visible=True),
                    name=str(tr), legendgroup=str(tr), showlegend=c == 0 and r == 0,
                    mode="lines+markers", line=dict(color=color),
                )
            )
        if c == 0:
            coverage_fig.update_yaxes(
                row=r + 1, col=c + 1,
                title=dict(text=row_title, font=dict(size=14, color='black'))
            )
del c, r, i, images_per_trial, col_title, row_title, prefix, tr, color, df

coverage_fig.update_layout(
    height=650, width=1500,
    title=dict(
        text="Coverage by Experimental Configuration", font=dict(size=20, color='black')
    ),
    legend=dict(
        title=dict(text="Trials per Subject"),
        orientation="h",
        x=1.0, xanchor="right", xref="paper",
        y=1.075, yanchor="bottom", yref="paper",
    ),
)
coverage_fig.show()

### Stability
We check the Spearman (rank) correlation between different iterations of the same experimental configuration to see how stable the results are.

In [ ]:
correlations = dict()
for param in sim._results.keys():
    corrs = []
    for res1, res2 in combinations(sim._results[param], 2):
        try:
            corr = metrics.spearman_correlation(res1, res2)
            corrs.append(corr)
        except ValueError:
            # if we don't have mutual observations we can't compute a correlation, so we skip those cases
            pass
    corrs = pd.Series(corrs)
    mean, sem = corrs.mean(), corrs.sem()
    correlations[param] = pd.Series({"r_mean": mean, "r_sem": sem})
del param, corrs, res1, res2, corr, mean, sem

correlations = pd.DataFrame(correlations).T
correlations.index.names = [
    'n_subjects', 'trials_per_subject', 'images_per_trial', 'subjects_noise_scale', 'subjects_noise_df'
]

In [ ]:
ROW_TITLES = {tr: f"{tr} Trials per Subject" for tr in coverage_summary["trials_per_subject"].unique()}
COL_TITLES = {k: f"{k} Images per Trial" for k in coverage_summary["images_per_trial"].unique()}
corr_fig = make_subplots(
    rows=len(ROW_TITLES), cols=len(COL_TITLES),
    column_titles=list(COL_TITLES.values()),
    shared_xaxes=True, shared_yaxes=True,
    x_title="Number of Subjects",
    vertical_spacing=0.05, horizontal_spacing=0.025,
)
for c, (images_per_trial, col_title) in enumerate(COL_TITLES.items()):
    for r, (trials_per_subject, row_title) in enumerate(ROW_TITLES.items()):
        subset = correlations.loc[
            (correlations.index.get_level_values("images_per_trial") == images_per_trial) &
            (correlations.index.get_level_values("trials_per_subject") == trials_per_subject)
        ]
        noise_scales = subset.index.get_level_values("subjects_noise_scale").unique()
        noise_dfs = subset.index.get_level_values("subjects_noise_df").unique()
        for i, (noise_scale, noise_df) in enumerate(product(noise_scales, noise_dfs)):
            name = f"Scale={noise_scale}<br>DFs={noise_df}"
            color = px_colors.qualitative.Plotly[i % len(px_colors.qualitative.Plotly)]
            df = subset.loc[
                (subset.index.get_level_values("subjects_noise_scale") == noise_scale) &
                (subset.index.get_level_values("subjects_noise_df") == noise_df)
            ]
            corr_fig.add_trace(
                row=r + 1, col=c + 1, trace=go.Scatter(
                    x=df.index.get_level_values("n_subjects"),
                    y=df["r_mean"],
                    error_y=dict(type="data", array=df["r_sem"], visible=True),
                    name=name, legendgroup=name, showlegend=c == 0 and r == 0,
                    mode="lines+markers", line=dict(color=color),
                )
            )
        if c == 0:
            corr_fig.update_yaxes(
                row=r + 1, col=c + 1,
                title=dict(text=row_title, font=dict(size=10, color='black'))
            )
del c, r, i, images_per_trial, col_title, trials_per_subject, row_title, noise_scale, noise_df, name, color, df

corr_fig.update_layout(
    height=650, width=1500,
    title=dict(
        text="Stability (Spearman Correlation) by Experimental Configuration",
        font=dict(size=20, color='black')
    ),
    legend=dict(
        title=dict(text="Subject Noise Parameters"),
    ),
)
corr_fig.show()

## Run MDS
In the real world, we don't know the true dimensionality of the data (like here, `D=5 or 10`). Instead, we run the MDS algorithm on the same distance matrix with increasing target-dimensionalities (`ndim=2, 3, ..., 10`).

In [ ]:
import gc
import pickle as pkl

from SpAM_Simulations.multi_dimensional_scaling import run_mds

In [ ]:
def mds_tasks_generator(sim_obj, max_dims):
    """ a generator-function to iterate over experimental configurations and their simulated results """
    for param, results_list in sim_obj._results.items():
        # iterate over remaining configurations' results
        for rep_idx, res in enumerate(results_list):
            for ndim in range(2, max_dims + 1):
                yield param, rep_idx, res, ndim


def load_mds_results(filepath):
    """
    Loads a pickle file containing multiple appended (key, value) pairs and reconstructs them into a single dictionary.
    """
    data_dict = {}
    if not (os.path.exists(filepath) or os.path.isfile(filepath)):
        print(f"File {filepath} not found.")
        return data_dict
    with open(filepath, "rb") as f:
        count = 0
        while True:
            try:
                # pickle.load reads the next object in the stream
                key, val = pkl.load(f)
                data_dict[key] = val
                count += 1
            except EOFError:
                # End of file reached
                break
            except Exception as e:
                print(f"Corrupted record at index {count}: {e}")
                break
    print(f"Successfully loaded {count} results into dictionary.")
    return data_dict

In [ ]:
GT_DIMS = sim.gt_embeddings.shape[1]    # number of dimensions in simulated database
NUM_TASKS = sum(1 for _ in mds_tasks_generator(sim, GT_DIMS))
MAX_ITERS = 500

In [ ]:
print(f"Running {NUM_TASKS} MDS tasks...")

i = 0
for exp_params, rep, exp_res, ndim in tqdm(mds_tasks_generator(sim, GT_DIMS), total=NUM_TASKS):
    i += 1
    try:
        mean_dists = exp_res.distances / exp_res.num_obs    # average over bumber of observations of the image-pair
        weights = (exp_res.num_obs > 0).astype(float)       # weight is 0 for missing image-pairs, 1 for observed
        mds_res = run_mds(
            dists=mean_dists,
            weights=weights,
            ndim=ndim,
            max_iters=MAX_ITERS,    # relax number of iterations
            convergence_tol=1e-6,
            precalc_init=False,
            verbose=False,
        )
        data_to_save = (mds_res["niter"], mds_res["stress"], mds_res["confdist"].round(6).astype(np.float32))
        del mds_res
    except Exception as e:
        data_to_save = e
    key_to_save = tuple({**exp_params._asdict(), "rep": rep, "ndim": ndim}.items())
    with open("mds_results.pkl", "ab") as f:
        pkl.dump((key_to_save, data_to_save), f)
    del data_to_save, key_to_save
    if i % 100 == 0:
        gc.collect()

In [ ]:
mds_results = load_mds_results("mds_results.pkl")

### Percent Failures
We want to check the percentage of MDS runs that failed for each experimental configuration and target dimensionality.<br>
We checked the number of connected components before, but here we check for other causes as well, for example:
- a RuntimeError with the words "connected component" indicates that there were more than 1 CC.
- a RuntimeError with the words "max_iter" indicates that the MDS algorithm failed to converge within the maximum number of iterations.

In [ ]:
from collections import Counter

reasons = dict()
for key, val in mds_results.items():
    if isinstance(val, Exception):
        if "connected components" in str(val):
            status = "connected components"
        else:
            status = str(val)
    elif val[0] == MAX_ITERS:
        status = "max_iters"
    else:
        status = "success"
    reasons[key] = status

Counter(reasons.values())

### Scree Plot
We plot the **Scree Plot** showing Stress vs Dimensionality, and look for the "elbow" in the plot to determine the optimal dimensionality.<br>
We should see if this is stable across iterations of the same experimental configuration.

In [ ]:
success_mds_results = []
for key, val in mds_results.items():
    if reasons[key] != "success":
        continue
    params = {p_name: p_val for (p_name, p_val) in key}
    mds_res = {"n_iter": val[0], "stress": val[1], "confdist": val[2]}
    success_mds_results.append({**params, **mds_res})
success_mds_results = pd.DataFrame(success_mds_results).sort_values(by=[
    "trials_per_subject", "images_per_trial",
    "subjects_noise_scale", "subjects_noise_df", "num_subjects",
    "rep", "ndim",
])

In [ ]:
ROW_TITLES = {tr: f"{tr} Trials per Subject" for tr in success_mds_results["trials_per_subject"].unique()}
COL_TITLES = {k: f"{k} Images per Trial" for k in success_mds_results["images_per_trial"].unique()}
stress_scree_fig = make_subplots(
    rows=len(ROW_TITLES), cols=len(COL_TITLES),
    column_titles=list(COL_TITLES.values()),
    shared_xaxes=True, shared_yaxes=True,
    x_title="DIMENSIONS",
    vertical_spacing=0.05, horizontal_spacing=0.025,
)

for c, (images_per_trial, col_title) in enumerate(sorted(COL_TITLES.items())):
    for r, (trials_per_subject, row_title) in enumerate(sorted(ROW_TITLES.items())):
        subset = success_mds_results.loc[
            (success_mds_results["images_per_trial"] == images_per_trial) &
            (success_mds_results["trials_per_subject"] == trials_per_subject)
        ]
        noise_scales = success_mds_results["subjects_noise_scale"].unique()
        noise_dfs = success_mds_results["subjects_noise_df"].unique()
        n_subjs = success_mds_results["num_subjects"].unique()
        for i, (n_subj, noise_scale, noise_df) in enumerate(product(n_subjs, noise_scales, noise_dfs)):
            name = f"Scale={noise_scale}<br>DFs={noise_df}<br>N={n_subj}"
            color = px_colors.qualitative.Plotly[i % len(px_colors.qualitative.Plotly)]
            df = (
                subset.loc[
                    (success_mds_results["subjects_noise_scale"] == noise_scale) &
                    (success_mds_results["subjects_noise_df"] == noise_df) &
                    (success_mds_results["num_subjects"] == n_subj)
                ]
                .groupby("ndim")["stress"]
                .agg(N="count", mean="mean", sem="sem",)
            )
            stress_scree_fig.add_trace(
                row=r + 1, col=c + 1, trace=go.Scatter(
                    x=df.index.get_level_values("ndim"),
                    y=df["mean"],
                    error_y=dict(type="data", array=df["sem"].fillna(0), visible=True),
                    name=name, legendgroup=name, showlegend=c == len(COL_TITLES)-1 and r == len(ROW_TITLES)-1,
                    mode="lines+markers", line=dict(color=color),
                )
            )
        if c == 0:
            stress_scree_fig.update_yaxes(
                row=r + 1, col=c + 1,
                title=dict(text=row_title, font=dict(size=10, color='black'))
            )
del c, r, i, images_per_trial, col_title, trials_per_subject, row_title, noise_scale, noise_df, name, color, df

stress_scree_fig.update_layout(
    height=650, width=1500,
    title=dict(
        text="MDS Stress (lower is better)",
        font=dict(size=20, color='black')
    ),
    legend=dict(
        title=dict(text="Subject Noise Parameters"),
    ),
)
stress_scree_fig.show()

## Evaluate Embeddings
### Stability
We want to check how "stable" our embeddings are across different iterations of the same experimental configuration. To do this, we calculate the Spearman (rank) correlation between the pairwise distances of the embeddings from different iterations of the same experimental configuration.<br>
<br>
_Why use the Spearman correlation?_<br>
- _Cosine Similarity_: We don't care about the angle between embeddings, just that we maintain their distances relative to each other. We should allow for scaling and rotation of the embeddings, which would change the cosine similarity but not the relative distances between points.<br>
- _Pearson Correlation_: We don't care about the exact distances between points, just that they maintain their relative distances to each other. We should allow for scaling and rotation of the embeddings, which would change the Pearson correlation but not the Spearman correlation.<br>
- _Procrustes Analysis_: This is a more complex method that involves finding the optimal scaling and rotation to align two embeddings before calculating their similarity. This could be a good option, but it is more computationally intensive and may not be necessary for our purposes. The Spearman correlation is a simpler and more interpretable metric that still captures the stability of the embeddings across iterations.<br>

In [ ]:
from scipy.stats import spearmanr

grouping_cols = [
    "trials_per_subject", "images_per_trial",
    "subjects_noise_scale", "subjects_noise_df", "num_subjects",
    "ndim",
]
embedding_corrs = dict()
for group, df in tqdm(success_mds_results.groupby(grouping_cols)["confdist"]):
    corrs = pd.Series([
        spearmanr(res1, res2).statistic for (res1, res2) in combinations(df.values, 2)
    ])
    embedding_corrs[group] = {"N": len(corrs), "r_mean": corrs.mean(), "r_sem": corrs.sem()}
embedding_corrs_df = pd.DataFrame(embedding_corrs).T
embedding_corrs_df.index.names = grouping_cols
embedding_corrs_df = embedding_corrs_df.reset_index().sort_values(by=grouping_cols)

In [ ]:
ROW_TITLES = {tr: f"{tr} Trials per Subject" for tr in embedding_corrs_df["trials_per_subject"].unique()}
COL_TITLES = {k: f"{k} Images per Trial" for k in embedding_corrs_df["images_per_trial"].unique()}
embedded_corr_fig = make_subplots(
    rows=len(ROW_TITLES), cols=len(COL_TITLES),
    column_titles=list(COL_TITLES.values()),
    shared_xaxes=True, shared_yaxes=True,
    x_title="DIMENSIONS",
    vertical_spacing=0.05, horizontal_spacing=0.025,
)
for c, (images_per_trial, col_title) in enumerate(COL_TITLES.items()):
    for r, (trials_per_subject, row_title) in enumerate(ROW_TITLES.items()):
        subset = embedding_corrs_df.loc[
            (embedding_corrs_df["images_per_trial"] == images_per_trial) &
            (embedding_corrs_df["trials_per_subject"] == trials_per_subject)
        ]
        noise_scales = embedding_corrs_df["subjects_noise_scale"].unique()
        noise_dfs = embedding_corrs_df["subjects_noise_df"].unique()
        n_subjs = embedding_corrs_df["num_subjects"].unique()
        for i, (n_subj, noise_scale, noise_df) in enumerate(product(n_subjs, noise_scales, noise_dfs)):
            name = f"Scale={noise_scale}<br>DFs={noise_df}<br>N={n_subj}"
            color = px_colors.qualitative.Plotly[i % len(px_colors.qualitative.Plotly)]
            df = subset.loc[
                    (embedding_corrs_df["subjects_noise_scale"] == noise_scale) &
                    (embedding_corrs_df["subjects_noise_df"] == noise_df) &
                    (embedding_corrs_df["num_subjects"] == n_subj)
                ]
            embedded_corr_fig.add_trace(
                row=r + 1, col=c + 1, trace=go.Scatter(
                    x=df["ndim"],
                    y=df["r_mean"],
                    error_y=dict(type="data", array=df["r_sem"].fillna(0), visible=True),
                    name=name, legendgroup=name, showlegend=c == len(COL_TITLES)-1 and r == len(ROW_TITLES)-1,
                    mode="lines+markers", line=dict(color=color),
                )
            )
        if c == 0:
            embedded_corr_fig.update_yaxes(
                row=r + 1, col=c + 1,
                title=dict(text=row_title, font=dict(size=10, color='black'))
            )
del c, r, i, images_per_trial, col_title, trials_per_subject, row_title, noise_scale, noise_df, name, color,

embedded_corr_fig.update_layout(
    height=650, width=1500,
    title=dict(
        text="MDS-Stability (Spearman Correlation) by Experimental Configuration",
        font=dict(size=20, color='black')
    ),
    legend=dict(
        title=dict(text="Subject Noise Parameters"),
    ),
)
embedded_corr_fig.show()

In [ ]:
embedding_corrs_df["r_mean"].describe()